In [3]:
import yfinance as yf

In [4]:
df = yf.download('CEG', start='2021-01-01', end='2022-06-02')
df.reset_index(inplace=True)
df.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-19,38.000000,43.000000,38.000000,42.000000,41.759682,129900
1,2022-01-20,40.950001,48.369999,40.950001,45.000000,44.742512,153400
2,2022-01-21,43.000000,49.950001,42.950001,45.000000,44.742512,329500
3,2022-01-24,44.919998,45.000000,41.880001,42.799999,42.555099,110000
4,2022-01-25,42.500000,44.919998,41.330002,41.897999,41.658260,23500


1 Failed download:
- CEG: Data doesn't exist for startDate = 946702800, endDate = 1640926800